In [ ]:
# STEP 1: Install dependencies
!pip install music21 tensorflow

# STEP 2: Upload MIDI files
from google.colab import files
import os

print(" Upload your MIDI files (.mid)...")
uploaded = files.upload()

midi_files = []
for filename in uploaded.keys():
    if filename.endswith('.mid') or filename.endswith('.midi'):
        new_name = filename.replace(" ", "_")
        os.rename(filename, new_name)
        midi_files.append(new_name)

# STEP 3: Extract notes/chords from MIDI
from music21 import converter, instrument, note, chord
import pickle

notes = []
for file in midi_files:
    try:
        midi = converter.parse(file)
        parts = instrument.partitionByInstrument(midi)
        if parts:  # if parts exist, use only one instrument
            elements = parts.parts[0].recurse()
        else:
            elements = midi.flat.notes
        for el in elements:
            if isinstance(el, note.Note):
                notes.append(str(el.pitch))
            elif isinstance(el, chord.Chord):
                notes.append('.'.join(str(n) for n in el.normalOrder))
    except Exception as e:
        print(f" Error processing {file}: {e}")

print(f" Extracted {len(notes)} notes/chords.")
if len(notes) == 0:
    print(" No notes extracted. Please upload different MIDI files.")
    import sys
    sys.exit()

with open("notes.pkl", "wb") as f:
    pickle.dump(notes, f)

#  STEP 4: Prepare sequences
import numpy as np
from keras.utils import to_categorical

sequence_length = 100
pitch_names = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(pitch_names)}

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)
n_vocab = len(pitch_names)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1)) / float(n_vocab)
network_output = to_categorical(network_output)

#  STEP 5: Build LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation

model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

#  STEP 6: Train the model (set small epoch for testing)
print(" Training the model...")
model.fit(network_input, network_output, epochs=1, batch_size=64)

#  STEP 7: Save model and encoder
model.save("music_generator_model.h5")
with open("note_encoder.pkl", "wb") as f:
    pickle.dump(note_to_int, f)

print(" Model trained and saved as music_generator_model.h5")


🎵 Upload your MIDI files (.mid)...


Saving AnyConv.com__Pirates of the Caribbean X Game of Thrones  PianoViolin Cover  Eshan Denipitiya & David Loke.midi to AnyConv.com__Pirates of the Caribbean X Game of Thrones  PianoViolin Cover  Eshan Denipitiya & David Loke (5).midi
Saving AnyConv.com__Yedhee - Video Song  Dhanush  Pavish  Anikha  GV Prakash  Amal C Ajith  Sruthy Sivadas #JNAK.midi to AnyConv.com__Yedhee - Video Song  Dhanush  Pavish  Anikha  GV Prakash  Amal C Ajith  Sruthy Sivadas #JNAK (3).midi
Saving AnyConv.com__Apple Beauty Full Video Song  Janatha Garage  Jr. NTR, Samantha, Mohanlal  DSP Hit Songs.midi to AnyConv.com__Apple Beauty Full Video Song  Janatha Garage  Jr. NTR, Samantha, Mohanlal  DSP Hit Songs (3).midi
✅ Extracted 9875 notes/chords.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🎶 Training the model...
153/153 ━━━━━━━━━━━━━━━━━━━━ 540s 4s/step - loss: 7.3438


✅ Model trained and saved as music_generator_model.h5
